In [1]:
import feedparser
import json
import pprint as pp
import pandas

In [2]:
class toi_feed_reader:
    __global_rss_links: dict   #A class variable because it is expected to be a universal constant, but we need to initailize it as run time to enable reading from a file
    __json_loc = '../prompts/toi_rss_links.json'

    def __init__(self, city_name: str):
        self.__cityname = city_name
        self.__feed_url = toi_feed_reader.__global_rss_links.get(city_name)
        self.__feed = feedparser.parse(self.__feed_url)
        #pp.pprint(self.__feed)

    def get_feed_entries(self) -> pandas.DataFrame:
        article_title = []
        article_link = []
        article_link_id = []
        article_date = []
        
        for entry in self.__feed.entries:
            article_title.append(entry.title)
            article_link.append(entry.link)
            article_link_id.append(entry.link[-13:][:9])
            article_date.append(entry.published)

        df = pandas.DataFrame({
                        'links': article_link, 
                        'title': article_title,
                        'link_id': article_link_id,
                        'link_date': article_date
        })
        df.insert(0, 'site_name', 'TOI')
        df.insert(1, 'sub_site_name', self.__cityname)
        return df
        
    @classmethod
    def load_links(cls):
        with open(f'{cls.__json_loc}', 'r') as fp:
            data = fp.read()
        fp.close()
        if(data.strip()=='' or data.strip()=='{}' or data.strip()=='[]'):        #File is empty and has no links
            cls.__global_rss_links = {}
        else:
            cls.__global_rss_links = json.loads(data)

    @classmethod
    def show_all_rss_links(cls):
        return cls.__global_rss_links

    

In [3]:
#one line tester

#toi_feed_reader.load_links()
#rdr = toi_feed_reader('Mumbai')
#df = rdr.get_feed_entries()
#pp.pprint(df['title'])